In [7]:
import numpy as np
import json
import glob


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# #spacy
import spacy
from nltk.corpus import stopwords

#visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


from gensim.models import TfidfModel


In [8]:
#prepping data
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    return data


def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [9]:
stopwords = stopwords.words("english") + ["go", "re", "just", "get", "want", "know", "so", "need", "knock", "look", "guy", "work", "say", "let", "come", "here", "make", "see",  "tell", "thing", "talk", "ve", "really"]


In [10]:
episodeData = load_data("cleaned.json")

plots = [episodeData[episode] for episode in episodeData]

In [11]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load("/Users/anishpalakurthi/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-3.6.0", disable=["parser", "ner"])
    texts_out = []
    for sent in texts:
        #contains metadata about the word
        doc = nlp(sent)
        #lemmatizes each word by appending allowed tokens from the doc object's metadata
        texts_out.append(
            " ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))
        
    for word in texts_out:
        if word in stopwords:
            texts_out.remove(word)
    return texts_out

print("path successfully hit")
lemmatized_texts = lemmatization(plots)
print(lemmatized_texts[0])

path successfully hit
corridor sperm bank so photon direct plane slit slit observe go slit ’ unobserved however ’ observe ’ leave plane hit target go slit agree ’ point ’ point think ’ good idea tee shirt excuse hang move finger phylum make au see capital idea ’ port au help high sperm bank ask maybe here think place fill thank right back take time finish puzzle wait think kid ’re semi - pro commit genetic fraud ’ guarantee sperm go generate high iq offspring think sister basic dna mix hostesse idea little extra money get fractional bandwidth apartment know yearn fast download ’ poor woman go pin hope sperm wind toddler use integral differential solve area curve ’m sure still love want leave ’ protocol leave know ’ve never renege proffer sperm let try walk see stair apartment building still mad sperm bank hear interesting stair really height single step off little millimetre people trip care millimetre seem right ’ true series experiment father break clavicle send boarding school resul

In [12]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
        
    return final

#preprocess our lemmatized keywords
data_words = gen_words(lemmatized_texts)

print(data_words[0][0:10])

['corridor', 'sperm', 'bank', 'so', 'photon', 'direct', 'plane', 'slit', 'slit', 'observe']


In [13]:
#bigrams and trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)


def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)


print(data_bigrams_trigrams[0])


['corridor', 'sperm', 'bank', 'so', 'photon', 'direct', 'plane', 'slit', 'slit', 'observe', 'go', 'slit', 'unobserved', 'however', 'observe', 'leave', 'plane', 'hit', 'target', 'go', 'slit', 'agree', 'point', 'point', 'think', 'good', 'idea', 'tee_shirt', 'excuse', 'hang', 'move', 'finger', 'phylum', 'make', 'au', 'see', 'capital', 'idea', 'port', 'au', 'help', 'high', 'sperm', 'bank', 'ask', 'maybe', 'here', 'think', 'place', 'fill', 'thank', 'right', 'back', 'take', 'time', 'finish', 'puzzle', 'wait', 'think', 'kid', 're', 'semi', 'pro', 'commit', 'genetic', 'fraud', 'guarantee', 'sperm', 'go', 'generate', 'high', 'iq', 'offspring', 'think', 'sister', 'basic', 'dna', 'mix', 'hostesse', 'idea', 'little', 'extra', 'money', 'get', 'fractional', 'bandwidth', 'apartment', 'know', 'yearn', 'fast', 'download', 'poor', 'woman', 'go', 'pin', 'hope', 'sperm', 'wind', 'toddler', 'use', 'integral', 'differential', 'solve', 'area', 'curve', 'sure', 'still', 'love', 'want', 'leave', 'protocol', 'l

In [14]:
#tfidf removal

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []

words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
        
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    
    corpus[i] = new_bow



In [15]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

#[index, frequency]
print(corpus[0][0:10])


[(0, 3), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [16]:
#generate LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word = id2word, num_topics=5, random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = "mmds", R = 30)
vis

/Users/anishpalakurthi/Library/Python/3.10/lib/python/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.097699  0.041781       1        1  61.468684
3      0.044996  0.040444       2        1  21.454386
1     -0.096928 -0.039727       3        1   6.849924
2     -0.072274  0.094670       4        1   6.084556
0      0.026507 -0.137168       5        1   4.142450, topic_info=       Term         Freq        Total Category  logprob  loglift
210      go  8961.000000  8961.000000  Default  30.0000  30.0000
417      re  8667.000000  8667.000000  Default  29.0000  29.0000
1227  knock  1407.000000  1407.000000  Default  28.0000  28.0000
207     get  5561.000000  5561.000000  Default  27.0000  27.0000
529    time  2696.000000  2696.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
552     use    49.061304  1105.356733   Topic5  -5.9616   0.0690
567     way    49.434913  1409.198297   Topic5  -5.9540  -0.1662
234    here    51.913341  2685.513409   Topic5  -5.9051  -0.7622
295     let    50.079955  1928.258863   Topic5  -5.9411  -0.4669
356     now    48.193165  2637.618904   Topic5  -5.9795  -0.8185

[449 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
9952       2  0.937644      active
1904       3  0.907273  adolescent
10213      3  0.938072       algae
8235       5  0.845565       alibi
3249       1  0.646994         all
...      ...       ...         ...
580        4  0.053380        work
580        5  0.050926        work
7466       1  0.407080      zombie
7466       2  0.045231      zombie
7466       4  0.520158      zombie

[819 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])